1. (https://news.ycombinator.com/item?id=14949392)
1. (https://stackoverflow.com/questions/4434292/catamorphism-and-tree-traversing-in-haskell)
1. (https://en.wikibooks.org/wiki/Haskell/Fix_and_recursion)
1. (https://stackoverflow.com/questions/41524960/recursion-schemes-using-fix-on-a-data-type-thats-already-a-functor)
1. (https://www.schoolofhaskell.com/user/bartosz/understanding-algebras)
1. (https://stackoverflow.com/questions/21688063/haskell-to-fix-or-not-to-fix)

`cata :: Functor f => (f a -> a) -> (Fix f -> a)`

Beware the difference between 

`_Y f = f (_Y f)`  (recursion, value--copying) and 

`fix f = x where x = f x ` (corecursion, reference--sharing).




(https://stackoverflow.com/questions/27856974/functor-instance-for-generic-polymorphic-adts-in-haskell#27857225)

If you have a polymorphic type, like a List or a Tree, you can create a functor from (Hask × Hask) to Hask that represents them. **b param normally is recursion**, for example:

In [14]:
data ListF a b = NilF | ConsF a b  -- L(A,B) = 1+A×B
data TreeF a b = EmptyF | NodeF a b b -- T(A,B) = 1+A×B×B

These types are polymorphic on A but are fixed points regarding B, something like this:

In [17]:
newtype Fix f = Fix { unFix :: f (Fix f) }
type List a = Fix (ListF a)
type Tree a = Fix (TreeF a)
--... to contunue

In [1]:
{-# LANGUAGE DeriveFunctor #-}

In [11]:
import Data.Fix --(cata, Fix)
data ExprF r = Const Int
             | Add r r
             | Mul r r
    deriving Functor

--newtype Fix f = Fx (f (Fix f))
--unFix :: Fix f -> f (Fix f)
--unFix (Fix x) = x

--cata :: Functor f => (f a -> a) -> Fix f -> a
--cata alg = alg . fmap (cata alg) . unFix

alg :: ExprF Int -> Int
alg (Const i)   = i
alg (x `Add` y) = x + y
alg (x `Mul` y) = x * y

eval :: Fix ExprF -> Int
eval = cata alg
-- show
testExpr = Fix $ 
             (Fix $ (Fix $ Const 2) `Add` (Fix $ Const 3)) `Mul` 
             (Fix $ Const 5) :: Fix ExprF

print $ eval testExpr

25

In [3]:
{-https://stackoverflow.com/questions/4434292/catamorphism-and-tree-traversing-in-haskell-}
data Tree a = Leaf a
            | Branch (Tree a) (Tree a)

{-I dont understand the structure between{} 
however it defines two morphisms, leaf and branch 
leaf take an a and returns an r, branch takes two r and returns an r-} 
data TreeAlgebra a r = TreeAlgebra { leaf   :: a      -> r
                                   , branch :: r -> r -> r }

{- foldTree is a morphism that takes: a TreeAlgebra for Tree a with result r, a Tree a
and returns an r -} 
foldTree :: TreeAlgebra a r -> Tree a -> r
foldTree a@(TreeAlgebra {leaf   = f}) (Leaf   x  ) = f x
foldTree a@(TreeAlgebra {branch = g}) (Branch l r) = g (foldTree a l) (foldTree a r)

t0 = Branch (Branch (Leaf 23) (Leaf 7)) (Leaf 11)

sumAlgebra :: TreeAlgebra Int Int
sumAlgebra = TreeAlgebra { leaf = id
                         , branch = (+) }
foldTree sumAlgebra t0

41

take the "fixed point" of a parameterized data type -- that is,
constructing a data type  T such that 

`T = TreeNode a T`

They call this operator Mu.

`type Mu f = f (Mu f)`

`type IntTree = Mu IntNode`

essential property of the catamorphism is that to compute some function f, it suffices to have the values of f for its immediate children

So to compute r on a particular node, we need the node with its children replaced with their rs

we can compute r for the entire recursive structure (remember recursion is denoted explicitly here with Mu):

`cata :: (node r -> r) -> Mu node -> r`

`cata :: (IntNode r -> r) -> IntTree -> r`

`fmap :: (a -> b) -> node a -> node b`

`cata :: (Functor node) => (node r -> r) -> Mu node -> r`

`cata f t = f (fmap (cata f) t)`

In [4]:
-- Let's make a data type for a single node in your tree, parameterized over its children:
data TreeNode a child
    = Leaf a
    | Branch child child -- just replaced the recursive children with a type of our choosin child == Tree a 

In [5]:
import Data.Fix(cata, Fix)

In [6]:
newtype Id a = Id a
newtype Bot = Bot (Data.Fix.Fix Id) -- equals          newtype Bot=Bot Bot
-- There is only one allowable term. 
--            Bot $ Bot $ Bot $ Bot ...
:t Bot

Bot :: Fix Id -> Bot

The Fix data type cannot model all forms of recursion. Take for instance this nonregular data type.

`data Node a = Two a a | Three a a a`

`data FingerTree a = U a | Up (FingerTree (Node a))`

It is not easy to implement this using Fix.

In [7]:
data Tree' a = Branch' (Tree' a) (Tree' a) | Leaf' a
data TreeNode a child = Branch child child | Leaf a
  deriving (Show, Functor)
--data Fix f = F (f (Fix f))
type Tree a = Fix (TreeNode a) -- deriving Functor
--newtype Tree a = Tree (Fix (TreeNode a)) -- deriving Functor
--newtype Tree a = Tree ( Fix (TreeNode a) ) deriving Functor


In [8]:
-- cata == Catamorphism or generic function fold.
:t cata 

cata :: forall a (f :: * -> *). Functor f => (f a -> a) -> Fix f -> a

In [9]:
--sumTree :: Tree Int -> Int
sumTree :: Fix (TreeNode Int) -> Int
sumTree = Data.Fix.cata algebra
  where
    algebra (Branch a b) = a + b
    algebra (Leaf a) = a

--foldFunction :: TreeAlgebra a r -> (TreeNode a r -> r)
--foldFunction alg (Leaf a) = leaf alg a
--foldFunction alg (Branch l r) = branch alg l r

--treeCata :: TreeAlgebra a r -> (Tree a -> r)
--treeCata alg = cata (foldFunction alg)    

In [10]:
t = Fix $ Leaf 17 :: Tree Integer
--t = Branch (Leaf 3) $ Branch (Leaf 7) (Leaf 17) :: Fix (TreeNode Int)
:t t
--sumTree $ Branch (Leaf 3) $ Branch (Leaf 7) (Leaf 17)
sumTree t

t :: Tree Integer

<interactive>:1:9: error:
    • Couldn't match type ‘Integer’ with ‘Int’
      Expected type: Fix (TreeNode Int)
        Actual type: Tree Integer
    • In the first argument of ‘sumTree’, namely ‘t’
      In the expression: sumTree t
      In an equation for ‘it’: it = sumTree t